# q84
84: 単語文脈行列の作成
83の出力を利用し，単語文脈行列XXを作成せよ

心が折れたのでここから100分の1コーパスを使った．

## 使ったライブラリなど
scipy.sparse: 疎行列を扱うためのライブラリ
cPickle: pickleのC実装．pickleよりもいくらか速い．cPickleのデメリット(サブクラス化できないとかなんとか)は理解していない．

## 方針
* q83で作成したf(t,c), f(t,*), f(*,c)らのtxtファイルをロードする．それぞれを辞書に格納．
* 疎行列のインデックスにwordを直接利用することはできないため，word_setを作ってからwordからidxに変換する辞書を作成する．（これは後にpickle化する）
* 問に与えられたPPMIの定義に沿って，疎行列に値を格納していく．0な値を作らないとかいうのは，疎行列のライブラリがうまくやってくれるはず．（値を挿入するまでは行列はメモリ上に展開されない，という雑な理解）
* 最後にcPickleを使って保存

In [1]:
import sys
import cPickle
import numpy as np
import scipy.sparse as sp
from collections import defaultdict

100分の1サイズのコーパスでq83を実行したときに得たNの値を決め打ちした．

In [2]:
N = 387858

In [3]:
tc_freq = defaultdict(lambda: defaultdict(int))
xc_freq = {}
tx_freq = {}
word_set = set()
with open("/work/kiyono/f_tc.txt", "r") as tc_file, open("/work/kiyono/f_c.txt", "r") as xc_file, open("/work/kiyono/f_t.txt", "r") as tx_file:
    for line in tc_file:
        target,context,count = line.rstrip().split("\t")
        tc_freq[target][context] = int(count)
    
    for line in xc_file:
        word,freq = line.rstrip().split("\t")
        word_set.add(word)
        xc_freq[word] = int(freq)
        
    for line in tx_file:
        word, freq = line.rstrip().split("\t")
        tx_freq[word] = int(freq)
    
    word2idx = {}
    for n, word in enumerate(word_set):
        word2idx[word] = n
        

In [4]:
matrix = sp.lil_matrix((len(word2idx), len(word2idx)))

In [5]:
for target in tc_freq.iterkeys():
    for context, count in tc_freq[target].iteritems():
        if count >= 10:
            t_idx = word2idx[target]
            c_idx = word2idx[context]
            matrix[t_idx,c_idx] = max(np.log((N*count) * 1.0/(xc_freq[context] * tx_freq[target] * 1.0)) , 0.0)
        else:
            pass

In [6]:
word_idx_file = open("./data/q84_word2idx.pkl", "w")
cPickle.dump(word2idx, word_idx_file)
word_idx_file.close()

In [7]:
matrix_file = open("./data/q84_matrix.pkl", "w")
cPickle.dump(matrix, matrix_file)
matrix_file.close()